<a href="https://colab.research.google.com/github/iamchenchu/Deep-Learnig-with-TensorFlow/blob/main/09_Natural_Language_Processing_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP(Naturala Language Processing) and NLU (Natural Language Understanding) :

`A handful of example natural language processing (NLP) and natural language understanding (NLU) problems. These are also often referred to as sequence problems (going from one sequence to another)`

NLP has the goal of deriving information out of natural language (Could be sequences text or speech). Another common term of NLP probelems is sequence to sequence problems (seq2seq)

**Let's see where do we use it : Which are reffered to as a Sequence Problems**

1. MultiLabel Classification
2. Machine Translation
3. Text Generation
4. Voice Assistant

**Types of Sequence problems :(input to output types)**
1. One to one  (one word translation)
2. One to many (Image captioning, possible to have multiple captions)
3. Many to one (Sentiment analysis maybe a youtube comment, BitCoin Price Prediction)
4. many to many (Part-of-speech tagging in natural language processing. Each word in the input sentence may correspond to multiple possible part-of-speech tags.)
5. many to many (DNA sequence alignment. Aligning two DNA sequences where each base in one sequence may correspond to multiple bases in the other.)







# What will we do in this section :         

* Downloading and preparing the text data
* How to prepare text data for modeling (Tokenization and embedding)
* Setting up multiple modeling experiments with Recurrent Neural Networks (RNN)
* Building a text feature extraction model using TensorFlow Hub
* Finding the most wrong prediction examples
* Using a model we've built to make predictions on the text from the wild



**NLP Inputs and outputs :**

`Text -> turn into numbers -> build a model -> train the model to find patterns -> use patterns (make predictions)`

esource: For a great overview of NLP and the different problems within it, read the article A Simple Introduction to [Natural Language Processing](https://becominghuman.ai/a-simple-introduction-to-natural-language-processing-ea66a1747b32).



# STEPS  WE FOLLOW IN CREATING IN RNN / CNN FOR NLP

1. Get the data ready (Turn all data into numbers as neural networks can't handle text or natural language and make sure all of your tensors are in right shape pad sequence which don't fit )
2. Build or pick a pretrained model (to suit your problem)
3. Fit the model to the data and make a prediction
4. Evaluate the model
5. Improve through Experimentation
6. Save and reload your trained model



# TYPICAL ARCHITECTURE OF A RECCURENT NEURAL NETWORK

**What is RNN :**  is class of Artificial Neural Networks where connections between nodes from a directed graph along a temporal sequence. This allows it to exhibit temporal dynamic behaviour.

1. Input Text
2. Input Layers
3. Text vectorization layer
4. Embedding (Turns mapping of text vectors to embedding matrix (representing how words relate Ex King-man+woman = queen))
5. RNN Cells (Finds Pattern in sequence) `Ex : Simple RNN, LSTM, GRU`
6. Hidden Activations : Adds non linearity to learned features usually use `tanh`
7. Pooling layer : Reduces the dimensionality of the learned features usually Conv1D models `Ex : GlobalAveragePooling1D or GlobalMaxPool1D`
8. Fully Connected layer : Further refines learned features from recurrent layers (Dense layer)
9. Output layer (Takes learned feature and outputs them in shape of target labels) Ex : `output_shape = [number_of_classes] (Disaster or not)`
10. Output Activation : Adds non-linearities to output layer.






# Check GPU

In [1]:
!nvidia-smi

Tue Jan  2 15:46:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Get helper functions

In [2]:
# Get helper functions Import the series of the helper functions using the below link

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#Import the functions using import
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-01-02 15:46:31--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-02 15:46:31 (133 MB/s) - ‘helper_functions.py’ saved [10246/10246]



# Get the Text Dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as diaster or not diaster)

See the original source here :https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip



In [3]:
#Get the data

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#Unzip the data
unzip_data("nlp_getting_started.zip")

--2024-01-02 15:46:35--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 64.233.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K   722KB/s    in 0.8s    

2024-01-02 15:46:36 (722 KB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



# Becoming one with the data and visualizing a text dataset

To visualize our text samples, we first have to read them in, one way to do so would be to use python: https://realpython.com/python-csv/#:~:text=Reading%20from%20a%20CSV%20file,which%20does%20the%20heavy%20lifting.


 but I prefer to get visual straight way

 So another way to do this to use pandas.



In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [6]:
#Shuffle training dataframe
train_df_shuffled = train_df.sample(frac =1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
len(train_df), len(test_df)

(7613, 3263)

In [8]:
#What does the test dataframe look like

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
#How many total samples
len(train_df) + len(test_df)

10876

In [10]:
#Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create a random indexes not higher than the total num
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target : {target}", "(real diaster)" if target >0 else "(not diaster)")
  print(f"Text :  {text}\n")
  print("---\n")

Target : 0 (not diaster)
Text :  @DorisMatsui thank you for supporting the President. The #IranDeal takes nuclear weapons out of the hands of Iran and keeps everyone safer.

---

Target : 0 (not diaster)
Text :  A new favorite: Desolate 2 by r3do https://t.co/HDv3ZirBCi on #SoundCloud

---

Target : 0 (not diaster)
Text :  @MsMigot wow what convincing &amp; compelling evidence to change my view that is. climate change deniers I tend to destroy by giving...EVIDENCE

---

Target : 1 (real diaster)
Text :  Hurricane 30STM quem lembra

---

Target : 1 (real diaster)
Text :  Flood and Mudslide Warning for East Fjords | Iceland Review http://t.co/534q3Jg2OV #icelandreview via @iceland_review

---



# Split data into training and validation sets



In [11]:
from sklearn.model_selection import train_test_split



In [12]:
# Use train_test_split to split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1, # use 10% of training data for validation
                                                                            random_state = 42)

In [13]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
#Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

# Converting text into numbers

When dealing with the text problems, one of the first things you'll have to do before you can build a model is to convert text to numbers.

`There are few ways to do this, namely :`  
**Tokenization :** Straight mapping from token to number (Can be modelled but quickly get's too big)

**Ebedding :** Richer representation of relationships between tokens (Can limit size + can be learned) : Create a matrix of feature vector for each token (the size of the feature vector can be defined and this embeding can be learned)





# Text Vectorization (Tokenization)

In [15]:
train_sentences[:5]


array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
# Text Vectorization (Tokenization)

import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#Use the default TextVectorization
text_vectorizer = TextVectorization(max_tokens=10000, # How many words in the vocabulary (automatically add  <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams = None, # Create groups of n-words
                                    output_mode = "int", # how to map tokens to numbers
                                    output_sequence_length = None, #how long do you want your sequences to be
                                    pad_to_max_tokens = True)

In [17]:
len(train_sentences[0].split())

7

In [18]:
#Find the average number of tokens (words) in the training tweets

round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
#Setup text vectorization variables
max_vocab_length = 10000 # Max number of words to have in our vocabulary
max_length = 15 # Max length our sequences will be

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length)

In [20]:
#Fit the text vectorizer to the training text

text_vectorizer.adapt(train_sentences)


In [21]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [22]:
#Create a sample sentence and tokenize it
sample_sentance = "There is a flood in my street"
text_vectorizer([sample_sentance])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
# Choose a random sentence from a training dataset and tokenize it

random_sentence = random.choice(train_sentences)
print(f"Original text : {random_sentence}\
        \n\nVectorized version :")
text_vectorizer([random_sentence])


Original text : All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected        

Vectorized version :


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  44, 2003, 1901,    5, 1983,    4,  596,   22,  121,    1,   18,
        1802,   40,  454,  245]])>

In [24]:
#Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get the all of the unique words in the
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocabulary : {len(words_in_vocab)}")
print(f"Top 5 words in vocabulary : {top_5_words}")
print(f"Bottom 5 words in vocabulary : {bottom_5_words}")

Number of words in vocabulary : 10000
Top 5 words in vocabulary : ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 words in vocabulary : ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Creating an Embedding using an Embedding Layer

Turns a positive integers (indexes) into dense vectors of fixed size

To make our embeddding,  we're going to use tensorflow layer and for more infor please visit the below link : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

The parameters we care most about  for our embedding layer :      

* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented 100 long.
* `input_length`  = Length of the sequences being passes to the embedding layer



In [25]:
from tensorflow.keras import layers

#Create a embedding layer
embedding = layers.Embedding(input_dim = max_vocab_length, # Set input shape
                             output_dim = 128, # Output shape
                             input_length = max_length) # How long is each input
embedding

In [26]:
#Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text : {random_sentence}\
      \n \n Embedded Version :" )

#Embed the random sentences (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text : Stu put beetroot in his cake and even lost to a mudslide      
 
 Embedded Version :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04641118,  0.00338342, -0.0498995 , ..., -0.02803612,
         -0.03727627, -0.01645118],
        [-0.00873389,  0.01710178, -0.00565239, ..., -0.03933871,
          0.02152934,  0.04445318],
        [-0.02636193,  0.01136378,  0.01909735, ..., -0.02820936,
         -0.0416559 ,  0.0312507 ],
        ...,
        [ 0.02896691,  0.04692658,  0.04357818, ..., -0.01371019,
          0.04218466,  0.03940913],
        [ 0.02896691,  0.04692658,  0.04357818, ..., -0.01371019,
          0.04218466,  0.03940913],
        [ 0.02896691,  0.04692658,  0.04357818, ..., -0.01371019,
          0.04218466,  0.03940913]]], dtype=float32)>

In [27]:
#Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04641118,  0.00338342, -0.0498995 , -0.01010539,  0.02019597,
         0.02429471,  0.04166758,  0.00777811,  0.02990589,  0.03802294,
         0.03150118,  0.02246041, -0.03629041, -0.01161964, -0.03179339,
        -0.00662946,  0.02614785,  0.02880967, -0.00284687,  0.00103619,
        -0.0413928 ,  0.03082098, -0.04816872, -0.01100768,  0.0014873 ,
         0.02831097, -0.03420539, -0.02349608,  0.03527499,  0.04389561,
         0.02066435, -0.03526037,  0.02433098, -0.03592566, -0.02892959,
         0.03622696, -0.02061912,  0.02492186,  0.03953273,  0.00703676,
         0.00737524, -0.0435704 ,  0.02034186,  0.0014746 , -0.01999949,
         0.02765048,  0.04036416,  0.01316788,  0.0342414 ,  0.01608822,
        -0.04114811,  0.01934561, -0.01694655,  0.00568289,  0.03137909,
         0.01161001,  0.03216228, -0.00988649,  0.02067265, -0.04197367,
        -0.00142039,  0.01418744,  0.04697062,  0.04664184,  0.0415544 ,
  

## Modeling a text dataset (Running a series of experiments)

Once you've got your inputs and outputs prepared, it's a matter of figuring out which machine learning model to build in between them to bridge the gap.

Now that we've got a way to turn our text data into numbers, we can start to build machine learning models to model it.

To get plenty of practice, we're going to build a series of different models, each as its own experiment. We'll then compare the results of each model and see which one performed best.

More specifically, we'll be building the following:

`Model 0`: Naive Bayes  TF-IDF (baseline) [Sklearn ML Map](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

`Model 1`: Feed-forward neural network (dense model)

`Model 2`: LSTM model

`Model 3`: GRU model

`Model 4`: Bidirectional-LSTM model

`Model 5`: 1D Convolutional Neural Network

`Model 6`: TensorFlow Hub Pretrained Feature Extractor (Using transfer learning for NLP)

`Model 7`: Same as model 6 with 10% of training data


Model 0 is the simplest to acquire a baseline which we'll expect each other of the other deeper models to beat.

**Each experiment will go through the following steps:**

*   Construct the model
*   Train the model
*   Make predictions with the model
*   Track prediction evaluation metrics for later comparison

      
      
      
      

# Model 0 : Getting a baseline

As with all machine learning modelling experiments, it's importat to create baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

**Note :** It's common practice to use non-DL algorithms as baseline because of their speed and then later using DL to see if you can improve upon them.



In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#Create tokenization and modeling pipeline
model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()), # Convert words to numbers using tfidf
    ('clf', MultinomialNB()) # model the text
])

#Fit the pipeline to training the data
model_0.fit(train_sentences, train_labels)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our Baseline model achieves an accuracy : {round(baseline_score * 100)}%")

Our Baseline model achieves an accuracy : 79%


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# Creating an evaluation function for our experiments

We could evaluate all of our model predictions with different metrics every time however, this would be cumbersome and could

Let's create 1 to compare our model's predictions with the truth labels using the following metrics :     

* Accuracy
* Precision
* Recall
* F-1 Score

For a deep overview of many different evaluation methods, see the sklearn documentation :     https://scikit-learn.org/stable/modules/model_evaluation.html


In [31]:
# Function to evaluate : Accuracy, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred) :
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100

  # Calculate model precision, recall, and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy ": model_accuracy,
                   "precision ": model_precision,
                   "recall ": model_recall,
                   "f1 ": model_f1}

  return model_results








In [32]:
#Get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred=baseline_preds)

baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

# Model 1: A Simple Dense Model (Feed-forward neural network)

In [33]:
# Create a tensorboard callback (Need to create a new one for each model)
from helper_functions import create_tensorboard_callback

#Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"




In [44]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
# x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [35]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [45]:
# Compile model
model_1.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ["accuracy"])

In [37]:
# Fit the model
model_1_history = model_1.fit(x = train_sentences,
                              y=train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240102-154639
Epoch 1/5
215/215 [==============================] - 9s 31ms/step - loss: 0.6102 - accuracy: 0.6908 - val_loss: 0.5361 - val_accuracy: 0.7480
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.4405 - accuracy: 0.8167 - val_loss: 0.4681 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3454 - accuracy: 0.8602 - val_loss: 0.4572 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2829 - accuracy: 0.8899 - val_loss: 0.4651 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2366 - accuracy: 0.9123 - val_loss: 0.4845 - val_accuracy: 0.7861


In [42]:
#Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4845 - accuracy: 0.7861


[0.48454439640045166, 0.7860892415046692]

In [39]:
#Make some predictions
model_1_pre_probs = model_1.predict(val_sentences)
model_1_pre_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [40]:
embedding.weights

[<tf.Variable 'embedding/embeddings:0' shape=(10000, 128) dtype=float32, numpy=
 array([[ 0.01564417,  0.06388038,  0.0617328 , ..., -0.0297114 ,
          0.02427213,  0.05672338],
        [-0.03352758,  0.02505742,  0.0334514 , ..., -0.04070355,
         -0.05508685,  0.04512011],
        [-0.0521352 ,  0.01880152, -0.00902492, ..., -0.03111177,
         -0.01763829,  0.02332321],
        ...,
        [ 0.03264341, -0.02792002, -0.00137975, ..., -0.04788015,
         -0.04354136, -0.02359537],
        [-0.04445776,  0.06801818,  0.03963505, ..., -0.04423115,
         -0.06870256,  0.01908221],
        [-0.11406244,  0.08413639,  0.0663091 , ..., -0.02238227,
         -0.02645211,  0.0755345 ]], dtype=float32)>]

In [43]:
# Make predictions (these come back in the form of probabilities)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 2ms/step


array([[0.30150828],
       [0.7862725 ],
       [0.99767727],
       [0.09128984],
       [0.09293614],
       [0.9366867 ],
       [0.91971505],
       [0.993129  ],
       [0.96028984],
       [0.2412541 ]], dtype=float32)

In [47]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [50]:
#COnvert model prediction probabilities to label formats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [51]:
#Calculate our model_1 results
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds)
model_1_results


{'accuracy ': 78.60892388451444,
 'precision ': 0.7925823788155513,
 'recall ': 0.7860892388451444,
 'f1 ': 0.7825463664667974}

In [52]:
baseline_results

{'accuracy ': 79.26509186351706,
 'precision ': 0.8111390004213173,
 'recall ': 0.7926509186351706,
 'f1 ': 0.7862189758049549}

In [53]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

# Visualizing learned embeddings